In [1]:
import tensorflow
import torch
from transformers import pipeline

In [3]:
# example of trying to classify items between cheese and animal
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
classifier(['brie', 'goat', 'cheddar', 'snake'],
           candidate_labels = ['cheese', 'animal']
)

[{'sequence': 'brie',
  'labels': ['cheese', 'animal'],
  'scores': [0.9708920121192932, 0.02910795621573925]},
 {'sequence': 'goat',
  'labels': ['animal', 'cheese'],
  'scores': [0.9980172514915466, 0.00198272499255836]},
 {'sequence': 'cheddar',
  'labels': ['animal', 'cheese'],
  'scores': [0.5735825300216675, 0.4264174699783325]},
 {'sequence': 'snake',
  'labels': ['animal', 'cheese'],
  'scores': [0.9993327260017395, 0.0006672403542324901]}]